# Notes

Second iteration is to build own input from the raw csv

every exercise in an entry
every entry consist of timeserieses for all face points 
a training/test split shall be based on patients so we could see a progression of exercises


In [1]:
import os 
import csv
from datetime import datetime
import pandas as pd
import numpy as np
import tensorflow as tf

# Data prep

In [2]:
def filename2metadata(filename):
    meta = filename.split(' ')
    patientid = meta[0]
    evaluation = int(meta[3].split('=')[1])
    poi = meta[2]
    flagbeforeoperation = meta[4].split('=')[1]
    exerciseinfo = meta[5].split('_')
    exerciseid = int(exerciseinfo[0])
    exercisename = exerciseinfo[1]
    exercisedatestring = exerciseinfo[2].split('.')[0]
    return {
        'patientid': patientid,
        'evaluation': evaluation,
        'poi': poi,
        'exerciseid': exerciseid,
        'exercisename': exercisename,
        'exercisedate': datetime.strptime(exercisedatestring, '%Y-%m-%d-%H-%M-%S'),
    }

In [3]:
def csv2timeseriestensor(df):
    xs = list(df['x'],)
    

In [4]:
def read2object(path, filename):    
    df = pd.read_csv(path)
    df.drop('t', axis=1, inplace=True)
    
    xs = np.transpose(df.to_numpy())

    return {
        'meta': filename2metadata(filename),
        'timeseries': xs
    }

TODO:
- read all
- padding
- split: normally split should be before padding, and then in inference combination of padding and trimming shall apply
- classification


In [5]:
data = []
maxlength = 0
dirdata = os.path.join('data', 'csv_points')
for filename in os.listdir(dirdata):
    datum = read2object(os.path.join(dirdata, filename), filename)
    data.append(datum)
    maxlength = max(maxlength, datum['timeseries'].shape[1])
    

maxlength

2047

In [6]:
len(data)

23748

In [7]:
maxlength = 2050

#### Padding

In [8]:
for ind, datum in enumerate(data):
    paddedtimeseries = tf.keras.preprocessing.sequence.pad_sequences(
        datum['timeseries'],
        padding="pre",
        maxlen=maxlength)
    data[ind]['timeseries'] = paddedtimeseries
    


#### Build training set

In [10]:
xslist = list()
yslist = list()

for datum in data:
    xslist.append(datum['timeseries'])
    yslist.append(datum['meta']['evaluation'])
    
ys = np.array(yslist[:23000])
xs = np.array(xslist[:23000])
ys_test = np.array(yslist[23000:])
xs_test = np.array(xslist[23000:])

In [11]:
print(ys.shape)
print(xs.shape)
print(ys_test.shape)
print(xs_test.shape)

(23000,)
(23000, 3, 2050)
(748,)
(748, 3, 2050)


In [12]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=xs[0].shape))
model.add(tf.keras.layers.LSTM(3))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(6, activation='softmax'))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'],
              run_eagerly=True)

model.summary()

2023-01-05 19:44:15.848384: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-05 19:44:15.848770: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 3)                 24648     
                                                                 
 dense (Dense)               (None, 128)               512       
                                                                 
 dense_1 (Dense)             (None, 6)                 774       
                                                                 
Total params: 25,934
Trainable params: 25,934
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(xs, ys, batch_size=64, epochs=5)

Epoch 1/5


2023-01-05 19:44:17.379367: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
/Users/alanchen/.local/share/virtualenvs/research-face-excercises-dynamics-o_HFBNSW/lib/python3.8/site-packages/keras/backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


360/360 [==============================] - 8s 21ms/step - loss: 1.1422 - accuracy: 0.4684
Epoch 2/5
360/360 [==============================] - 8s 21ms/step - loss: 1.0861 - accuracy: 0.4758
Epoch 3/5
360/360 [==============================] - 8s 21ms/step - loss: 1.0928 - accuracy: 0.4758
Epoch 4/5
360/360 [==============================] - 8s 21ms/step - loss: 1.0944 - accuracy: 0.4758
Epoch 5/5
360/360 [==============================] - 8s 21ms/step - loss: 1.0957 - accuracy: 0.4758


In [14]:
model.evaluate(xs_test, ys_test)

24/24 [==============================] - 0s 14ms/step - loss: 1.1054 - accuracy: 0.4733


[1.1053643226623535, 0.4732620418071747]

In [15]:
y_test_prob = model.predict(xs_test)

24/24 [==============================] - 0s 6ms/step


In [16]:
y_test_prob

array([[1.9036078e-04, 5.9997696e-01, 1.2883082e-01, 1.5891016e-01,
        5.1801972e-02, 6.0289729e-02],
       [2.1950775e-04, 5.9706056e-01, 1.2901311e-01, 1.5857264e-01,
        5.3225517e-02, 6.1908718e-02],
       [1.9035055e-04, 5.9998113e-01, 1.2883171e-01, 1.5891126e-01,
        5.1799171e-02, 6.0286459e-02],
       ...,
       [1.9035685e-04, 5.9996456e-01, 1.2883602e-01, 1.5891658e-01,
        5.1804069e-02, 6.0288481e-02],
       [1.9035685e-04, 5.9996456e-01, 1.2883602e-01, 1.5891658e-01,
        5.1804069e-02, 6.0288481e-02],
       [1.9035685e-04, 5.9996456e-01, 1.2883602e-01, 1.5891658e-01,
        5.1804069e-02, 6.0288481e-02]], dtype=float32)

__END__